# GMNS to AequilibraE example

## Inputs
Nodes as a .csv flat file in GMNS format
omx skim files with numbers of trips
omx skim file with distnace


## Steps
1.  Read the GMNS nodes, place centroids in dataframe

2. Read the trips

4. Generate new dataframe of O-D pairs


In [7]:
#!/usr/bin/env python
# coding: utf-8
print('Scott Smith: Python code to extract selected zone pairs from a person trip table \n')
import os
from os.path import join 
import numpy as np
import pandas as pd
import sqlite3
#import shutil  # needed?
import openmatrix as omx
import math
run_folder = 'C:/Users/Scott/Documents/Work/GMNS/ATL'  
highest_centroid_node_number = 6031

Scott Smith: Python code to extract selected zone pairs from a person trip table 



In [50]:
def sl_dist(lat1, lon1, lat2, lon2):
    from math import radians, cos, sqrt, pow
    mile_per_lat = 69.0
    mile_per_lon = mile_per_lat * math.cos(math.radians((lat1+lat2)/2.0))
    latdiff = mile_per_lat * (lat1-lat2)
    londiff = mile_per_lon * (lon1-lon2)
    return sqrt(pow(latdiff,2) + pow(londiff,2))


In [51]:
#print(sl_dist(40,-75,41,-76))

## Read the nodes, and set up the dictionary of centroids
The dictionary of centroids is used later in setting up the omx trip table

In [52]:
#Read the nodes
node_csvfile = os.path.join(run_folder, 'GMNS_node.csv')
df_taz = pd.read_csv(node_csvfile)   #data already has headers
print(df_taz.head())   #debugging
df_size = df_taz.shape[0]
print(df_size)
#print(df_node.loc[700,"node_type"])

   node_id  name    x_coord    y_coord  z_coord node_type   ctrl_type  \
0        1   NaN -84.416355  34.076861      NaN  centroid  no_control   
1        2   NaN -84.410110  34.067103      NaN  centroid  no_control   
2        3   NaN -84.398224  34.080752      NaN  centroid  no_control   
3        4   NaN -84.374950  34.077750      NaN  centroid  no_control   
4        5   NaN -84.364269  34.082443      NaN  centroid  no_control   

   zone_id  parent_node_id  
0      NaN             NaN  
1      NaN             NaN  
2      NaN             NaN  
3      NaN             NaN  
4      NaN             NaN  
63827


In [53]:
# drop the non centroids
index = df_taz[df_taz['node_type'] != 'centroid'].index
df_taz.drop(index, inplace=True)
print(df_taz.head())   #debugging
df_size = df_taz.shape[0]
print(df_size)

   node_id  name    x_coord    y_coord  z_coord node_type   ctrl_type  \
0        1   NaN -84.416355  34.076861      NaN  centroid  no_control   
1        2   NaN -84.410110  34.067103      NaN  centroid  no_control   
2        3   NaN -84.398224  34.080752      NaN  centroid  no_control   
3        4   NaN -84.374950  34.077750      NaN  centroid  no_control   
4        5   NaN -84.364269  34.082443      NaN  centroid  no_control   

   zone_id  parent_node_id  
0      NaN             NaN  
1      NaN             NaN  
2      NaN             NaN  
3      NaN             NaN  
4      NaN             NaN  
6031


In [54]:
# Set up the dictionary of centroids
# Assumption: the node_type = 'centroid' for centroid nodes
# The centroid nodes are the lowest numbered nodes, at the beginning of the list of nodes,
# but node numbers need not be consecutive
tazdictrow = {}
for index in df_taz.index:
    if df_taz['node_type'][index]=='centroid':
        #DEBUG print(index, df_node['node_id'][index], df_node['node_type'][index])
        tazdictrow[df_taz['node_id'][index]]=index
taz_list = list(tazdictrow.keys())
matrix_size = len(tazdictrow)   #Matches the number of nodes flagged as centroids
print(matrix_size)   #DEBUG
highest_centroid_node_number = max(tazdictrow, key=tazdictrow.get)  #for future use
print(highest_centroid_node_number)  #DEBUG

6031
6031


In [55]:
# Read the trip table and skim
dembf = omx.open_file(join(run_folder, '0_tntp_data' ,'demand.omx'),'r')
input_demand = np.array(dembf['matrix'])
print('sum of trips',np.sum(input_demand))

spbf = omx.open_file(join(run_folder, '2_skim_results','sp_skim.omx'),'r')
print('SP BASE SKIM FILE Shape:',spbf.shape(),'   Tables:',spbf.list_matrices(),'   Mappings:',spbf.list_mappings())
spbt = spbf['free_flow_time']
spbd = spbf['distance']
sp_dist = np.array(spbd)

outdebugfile = open(os.path.join(run_folder,'debug_demand2.txt'),"w")


sum of trips 1827526.0
SP BASE SKIM FILE Shape: (6031, 6031)    Tables: ['distance', 'free_flow_time']    Mappings: ['main_index']


In [57]:
for i in range(100):  #matrix_size)
    for j in range(matrix_size):
        if(input_demand[i][j]>0):
            lon1 = df_taz.loc[i,'x_coord']
            lat1 = df_taz.loc[i,'y_coord']
            lon2 = df_taz.loc[j,'x_coord']
            lat2 = df_taz.loc[j,'y_coord']
            sp_distance = sp_dist[i][j]
            sl_distance = sl_dist(lat1,lon1,lat2,lon2)
            print(i,j,lat1, lon1, lat2, lon2,sp_distance,sl_distance,file=outdebugfile)

In [58]:
outdebugfile.close()
dembf.close()
spbf.close()